In [1]:
from IPython.display import HTML

HTML('''<script>
hide_code=true; 
function code_toggle() {
   if (hide_code){
       $('div.input').hide();
   } else {
       $('div.input').show();
   }
   hide_code = !hide_code
} 
$( document ).ready(code_toggle);
</script>
NOTE: the code in this notebook is hidden for better readability.  
To toggle on/off, click <a href="javascript:code_toggle()">here</a>.''')

# <center>Proposal – Data Science Practicum I </center>
## <center>Dstl Satellite Imagery Feature Detection Problem: </center>


My work and educational background is in environmental sciences and I have experience in GIS and remote sensing.  Given my background, I want to do a visualization problem that I can potentially use as an example for potential employers.  On kaggle, I found the Dstl Satellite Imagery Feature Detection Problem that I would like to tackle.  Several 1km x 1km Satellite Images have been supplied and the objective is to detect and classify the following objects found on the satellite images:

* Buildings – large buildings, residential, non-residential, fuel storage facilities, fortified buildings
* Misc. Man-made structures
* Roads
* Tracks – poor/dirt/cart tracks, footpaths/trails
* Trees – woodlands, hedgerows, groups of trees, standalone trees
* Crops – contour plowing/croplands, grain (wheat) crops, row (potatoes, turnips) crops
* Waterways
* Standing water
* Large Vehicle – large vehicles (e.g. lorry, trucks, buses), logistics vehicles
* Small Vehicle – cars, vans, motorbikes

A discription of the problem can be found here:  www.kaggle.com/c/dstl-satellite-imagery-feature-detection
Using Python, I will first use different types of classification methods such as SVM and k-mean to detect the objects.  I will compare the classifications and determine which method worked best. The classifications will be performed on each image-pixel this means that features such as object shapes, size and texture will not be accounted for when analyzing the satellite images.  To account for such features, it might be better to use deep learning.  Therefore if time permits a classification model will be performed using deep learning to determine if better results will be obtained using this method.

### DATA:

The Dstl Satellite Imagery Feature Detection Problem supplies you with 1km x 1km satellite images of 360 different locations taken using the WorldView-3 Satellite.  WorldView-3 takes multispectral and Panchromatic images of a region .  Multispectral images capture image data at specific frequencies across the electromagnetic spectrum.  Panchromatic images, on the other hand, are images that are sensitive to a wide range of wavelengths of light.

![](worldview-3-spectral-bands.jpg?raw=true)

Two types of imagery spectral content are provided, 3-band and 16-band images.  The 3-band consist of the RGB color bands with a resolution of ~0.3m.  The 16 band images consist of 8-Multispectral band images with a resolution of 1.24m, 8-SWIR (Short Wave Infrared) band images with a delivered resolution of ~7.5m and Panchromatic images with a resolution of 0.31m.  Details about the images are given in the table below:


|**8 Multispectral band – resolution 1.24m**||
|---------------------------------------|
|Coastal:|                  400 - 450 nm|
|Blue: |450 - 510 nm|
|Green: |510 - 580 nm|
|Yellow: |585 - 625 nm|
|Red: |630 - 690 nm|
|Red Edge: |705 - 745 nm|
|Near-IR1: |770 - 895 nm|
|Near-IR2: |860 - 1040 nm|
|**8 SWIR - delivered resolution ~7.5m**||
|SWIR-1: |1195 - 1225 nm|
|SWIR-2: |1550 -1590 nm|
|SWIR-3: |1640 -1680 nm|
|SWIR-4: |1710 -1750 nm|
|SWIR-5: |2145 - 2185 nm|
|SWIR-6: |2185 - 2225 nm|
|SWIR-7: |2235 - 2285 nm|
|SWIR-8: |2295 - 2365 nm|
|**Panchromatic – resolution 0.31m**||
|Panchromatic: |450 - 800 nm|

In addition to satellite images, training data for 25 of the image locations is provided.  The training data are georeferenced polygon files of classified objects found in the 25 different images.

### DATA SOURCE:
The data for this problem can be downloaded at:
www.kaggle.com/c/dstl-satellite-imagery-feature-detection/data

|File Name|Available Formats|
|----------------|
|sample_submission.csv|.zip (14.89 kb)|
|grid_sizes.csv|.zip (2.17 kb)|
|sixteen_band|.zip (7.30 gb)|
|three_band|.zip (12.87 gb)|
|train_geojson_v3|.zip (14.22 mb)|
|train_wkt_v4.csv|.zip (11.08 mb)|

### ANALYZES

The supplied WorldView-3 satellite images contains multispectral bands.  These bands contain data that might not be visible to the human eye.  For example, looking at the RGB image below-left we see a football field with grass.  However when examaning the image using near-Infrared we can determine that the field is actually made of artificial turf.

![](9788139_orig.png?raw=true)

As all the bands give different information, we want to use the information off all these bands to try and classify objects found in the Satellite images.  However, the 3-band data that was supplied is pan-sharpened data created from the Panchromatic Images and the RGB Layers Bands.  I will therefore not use this data for the analysis only use the 16-band data that was given.
The three different types of images all have a different resolution.  For data to be analyzed using traditional classification methods, The images would have to be resampled to match resolutions.  In order to focus on the analysis part of the project, I will first solely look at the 8-Multispectral image.  When this Analysis is complete, I will go back, resample the images to matching resolutions and perform the analysis with 17 features (8-Multispectral images, 8-SWIR images, Panchromatic Images)

***Week 1 - Jan 7:***
The first step of the analyzes using python is to convert the 25 satellite images that have test data to a data matrix that contains row, column and band data for each pixel.  I will first import the images (GeoTiffs format) using Gdal and then convert the images to a Numpy array.  I will then reshape the Numpy arrays to a data matrix and concatanate all the band data so that everything is contained in one matrix
The next step will be to import the polygon files (Geojson format) and to rasterize the files.  Both processes will be done using Gdal.  After the files have been rasterized, they will be converted to Numpy arrays, reshaped to a data matrix. 






In [12]:
import os
import numpy as np
import csv
import math
import ogr
from osgeo import gdal, osr, gdalconst

#---------------------FUNCTION: Import satellite images-----------------------------------------------------------------#


def importsatimage(imagename):
    
    imagenameM = imagename + "_M.tif"    
    print imagenameM
    satM = gdal.OpenShared(os.path.join('sixteen_band/',imagenameM),gdalconst.GA_Update)
    
        # Open the data source and read in the extent
    
    if os.path.isfile('sixteen_band/'+imagenameM):
        satM = gdal.OpenShared(os.path.join('sixteen_band/',imagenameM),gdalconst.GA_Update)
    else:
        raise IOError('Could not find file ' + imagenameM)
    if satM is None:
        raise IOError('Could open file ' + str(satM))
        
    return satM

#--------------------FUNCTION: Convert Geotiff files to numpy arrays, resize the arrays same size----------------------#

def geotiff2numpy(sat, dir):

    L1 = np.array(sat.GetRasterBand(1).ReadAsArray(),dtype=np.int16)
    L2 = np.array(sat.GetRasterBand(2).ReadAsArray(),dtype=np.int16)
    L3 = np.array(sat.GetRasterBand(3).ReadAsArray(),dtype=np.int16)
    L4 = np.array(sat.GetRasterBand(4).ReadAsArray(),dtype=np.int16)
    L5 = np.array(sat.GetRasterBand(5).ReadAsArray(),dtype=np.int16)
    L6 = np.array(sat.GetRasterBand(6).ReadAsArray(),dtype=np.int16)
    L7 = np.array(sat.GetRasterBand(7).ReadAsArray(),dtype=np.int16)
    L8 = np.array(sat.GetRasterBand(8).ReadAsArray(),dtype=np.int16)
 
    #convert bands in XYZ format
    
    row,col = L1.shape
    R,C = np.mgrid[:row,:col]
    
    name = np.full((row*col, 1), dir, dtype=np.dtype('a8'))
    nameCR = np.column_stack((name, C.ravel(),R.ravel()))

    XYLbands = np.column_stack((L1.ravel(),L2.ravel(),L3.ravel(),L4.ravel(),L5.ravel(),L6.ravel(),L7.ravel(),L8.ravel() ))

    return XYLbands, nameCR

#----------------------FUNCTION: Import polygon files and reproject them-----------------------------------------------#

def importpolygons(dir, file, M_col, M_row, x_max, y_min):
    
    polygonfilename = 'train_geojson_v3/' + dir + '/' + file

    # Define pixel_size and NoData value of new raster

    NoData_value = 255

    # Open the data source and read in the extent
    
    polygonfile = ogr.Open(polygonfilename)
    
    if os.path.isfile(polygonfilename):
        polygon = ogr.Open(polygonfilename)
    else:
        raise IOError('Could not find file ' + polygonfilename)
    if polygon is None:
        raise IOError('Could open file ' + str(polygon))
    
    polygon_layer = polygon.GetLayer()
    polygon_srs = polygon_layer.GetSpatialRef()
    x_min = 0
    y_max = 0
    
    if x_max == 999:
        x_min, x_max, y_min, y_max = polygon_layer.GetExtent()
    pixel_sizex = x_max/M_col
    pixel_sizey = y_min/M_row

    # Create the destination data source
    
    x_res = int((x_max - x_min) / pixel_sizex)
    y_res = -1*int((y_max - y_min) / pixel_sizey)
    target_ds = gdal.GetDriverByName('MEM').Create('', x_res, y_res, gdal.GDT_Byte)
    target_ds.SetGeoTransform((0, pixel_sizex, 0, 0, 0, pixel_sizey))
    band = target_ds.GetRasterBand(1)
    band.SetNoDataValue(NoData_value)

    # Rasterize the Polygon file
    
    gdal.RasterizeLayer(target_ds, [1], polygon_layer, burn_values=[1])
    
    array = band.ReadAsArray()
    classified = array.ravel()
    
    return classified, x_max, y_min


#----------------------------------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------------------------------#


traindir ='./train_geojson_v3/'
data_matrix = None
classification_matrix = None
reference = {}
satsize = {}

with open('TRAINING_REFERENCE.csv') as csvfile:
    reader = csv.DictReader(csvfile)   
    for row in reader:
        reference[row['NAME']] = int(row['UNIQUEID'])

# read which images have test data

for parentdir, dirs, _ in os.walk(traindir):

# read in all satellite images that have test data and store the pixels in a data_matrix
    for dir in dirs:
        
        ImgM = importsatimage(dir)
            
        M_row = ImgM.RasterYSize
        M_col = ImgM.RasterXSize
        satsize[dir] = M_row, M_col
        
# Create a data matrix with, row, column, and the satellite band information for each pixel        

        if data_matrix is None:
 
            data_matrix, nameCR = geotiff2numpy(ImgM, dir)
    
        else:

            add2datamatrix, add2name = geotiff2numpy(ImgM, dir)
            data_matrix = np.concatenate((data_matrix, add2datamatrix),axis=0)
            nameCR = np.concatenate((nameCR, add2name),axis=0)

# Read in all classified polygon files and convert them to a data matrix

        for _, _, files in os.walk(os.path.join(parentdir, dir)):
            
            files.sort(reverse=True)

            class_matrix = np.zeros((M_col*M_row,5),dtype=np.int16)
    
            for file in files:              
                                
                if 'Grid' in file:

                    gridarray, x_max, y_min = importpolygons(dir, file, M_col, M_row, 999, -999)

                elif 'VEG' in file:
                    
                    ID = reference[file]-18
                    class_matrix[:,ID], x, y = importpolygons(dir, file, M_col, M_row, x_max, y_min)

                else: 

                    break

# Create a data matrix with, row, column, and the satellite band information for each pixel        

            if classification_matrix is None:
 
                classification_matrix = class_matrix
    
            else:

                classification_matrix = np.concatenate((classification_matrix, class_matrix),axis=0)
                
            #print classification_matrix.shape

6070_2_3_M.tif
6100_2_2_M.tif
6140_1_2_M.tif
6040_1_0_M.tif
6160_2_1_M.tif
6010_1_2_M.tif
6040_1_3_M.tif
6170_2_4_M.tif
6110_4_0_M.tif
6040_2_2_M.tif
6110_1_2_M.tif
6100_1_3_M.tif
6170_0_4_M.tif
6100_2_3_M.tif
6170_4_1_M.tif
6120_2_2_M.tif
6140_3_1_M.tif
6090_2_0_M.tif
6010_4_2_M.tif
6120_2_0_M.tif
6150_2_3_M.tif
6040_4_4_M.tif
6010_4_4_M.tif
6110_3_1_M.tif
6060_2_3_M.tif


***Weeks 2 & 3 – Jan 21:***
After the data has been imported and formatted, we will use sklearn to analyze the data.  The data will first be split into a training and testing data set.  The training data tset will be used to create classification models using sklearn.  After we have created the models we will cross-validate them using the testing data, and I will calculate the training and test error.  For each analyses a model , PR curves, ROC curves, and confusion matrixes will be created.

***Week 4 – Jan 28:***
After the classification we will convert classified data into geotiff images and overly them onto the satellite images.

***Week 5 – Feb 4:***
Once the classification on the 8-Multispectrum images has been completed, I will resample the 8-Multispectral and 8-SWIR images to match the resolution of the Panchromatic Images.  I will then follow the steps laid out above and perform a classification on the 17 feature data set (8-Multispectral images, 8-SWIR images, Panchromatic Images) .